## estrazione features da reti pretrainate

In [1]:
import numpy as np
import collections
import SimpleITK as sitk
from scipy.ndimage import zoom
import nrrd
import os,sys
import pandas as pd
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import Model


In [10]:

# caricamento modello
base_model = ResNet50(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.layers[142].output)

pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# funzione per creazione maschera
def maskcroppingbox(images_array, use2D=False):
    images_array_2 = np.argwhere(images_array)
    
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)
        
def featureextraction(image_array,mask_array):
    # ridimensionamento
    (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array, use2D=False)
    roi_images = image_array[zstart-1:zstop+1,ystart:ystop,xstart:xstop].transpose((2,1,0))
    roi_images1 = zoom(roi_images, zoom=[224/roi_images.shape[0], 224/roi_images.shape[1],1], order=3)
    roi_images2 = np.array(roi_images1,dtype=float)    

    # preprocessing per resnet
    x = image.img_to_array(roi_images2)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x = np.transpose(x, (3, 1, 2, 0))
    x = np.repeat(x, 3, axis=-1)

    # estrazione features principali come feature map
    base_model_pool_features = model.predict(x)
    
    feature_map = base_model_pool_features[0]

    feature_map = feature_map.transpose((2,1,0))
    features = np.max(feature_map,-1)
    features = np.max(features,-1)
    deeplearningfeatures = collections.OrderedDict()
    for ind_,f_ in enumerate(features):
    	deeplearningfeatures[str(ind_)] = f_
         
    return deeplearningfeatures


# salvataggio features in un file
featureDict = {}
for s in os.listdir(pathdicom):
    print(s)
    filename = os.path.join(pathdicom, s)

        
    for t in os.listdir(filename):

        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):

        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    
    deeplearningfeatures = featureextraction(readdatadicom,readdatanrrd) 

    result = deeplearningfeatures
    key = list(result.keys())
    key = key[0:]
        
    feature = []
    for jind in range(len(key)):
        feature.append(result[key[jind]])
        
    featureDict[s] = feature
    dictkey = key
    print(s)
    
dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\Features_Resnet50_142layer.csv')
    



100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 883ms/step
100
101
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 239ms/step
101
102
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 420ms/step
102
103
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 287ms/step
103
104
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 274ms/step
104
105
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step
105
106
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 540ms/step
106
107
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step
107
108
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 507ms/step
108
109
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 244ms/step
109
110
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 326ms/step
110
111
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step
111
112
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 296ms/step
112
113
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step
113
114
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step
114
115
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 264ms/step
115
116
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 351ms/step
116
117
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 207ms/step
117
118
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
118
119
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 393ms/step
119
12
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 364ms/step
12
120
2/2 ━━━━━━━